# Part 1: The Base Integer Program Formulation

#### Solve the problem in Python as an integer program according to the following guidelines (Objective & Constraints):  
  
a. Assume the objective is to maximize the total expected margin minus the cost of the 
samples for the next round of boxes.  
b. 5 items need to be sent to each customer.  
c. You can’t run out of inventory.  
d. You can’t send a customer a sample they’ve already received.  
  
Save the solution as an .xlsx file. Note how to interpret the solution and what the expected profit is 
under your optimal solution

In [1]:
# Import packages
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [4]:
# Reading in the itemdata and convert some columns to list or list of lists
itemdata = pd.read_excel(open('Birchbox_data.xlsx','rb'), sheet_name='item info') 
sku = itemdata.loc[:,"Item"].values.tolist()
inventory = itemdata.loc[:,"Inventory"].values.tolist()
margin = itemdata.loc[:,"Margin if Purchase"].values.tolist()
samplecost = itemdata.loc[:,"Sample Cost"].values.tolist()
seg_prob1 = itemdata.loc[:,"PurchaseProb_Seg_0":"PurchaseProb_Seg_13"].values.tolist()

In [6]:
# Reading in the custdata and do the same as the last cell
custdata = pd.read_excel(open('Birchbox_data.xlsx','rb'), sheet_name='cust info')
itemreceived = custdata.loc[:,"Item 0":"Item 110"].values.tolist()
cust_id = custdata.loc[:,"Cust #"].values.tolist()
segment = custdata.loc[:,"Seg"].values.tolist() #The Segment for each customer j
#seg_prob1[0][segment[0]] # The probability of customer 0 buying item 0
#len(itemreceived)

In [7]:
# Declare a concrete model
model = ConcreteModel()
num_item = 111 #possible items, (columns)index them by i
num_cust = 1047 #possible customers, (rows) index them by j

# Declare the decision variables
model.x = Var(range(num_item),range(num_cust), domain=Binary) #Which items are assigned to each customer?

# Constraint 1: each customer receive exactly 5 samples
model.sample = ConstraintList()
for j in range(num_cust):
    model.sample.add(expr = sum(model.x[i,j] for i in range(num_item)) == 5)
    
# Constraint 2: Cannot run out of inventory
model.inventory = ConstraintList()
for i in range(num_item):
    model.inventory.add(expr = sum(model.x[i,j] for j in range(num_cust)) <= inventory[i])
    
# Constraint 3: Cannot send a customer a sample they’ve already received.
model.uniquesample = ConstraintList()
for j in range(num_cust):
    for i in range(num_item):
        model.uniquesample.add(expr = (model.x[i,j] + itemreceived[j][i]) <=1)
        
# Objective calculation
expected_margin=sum(sum(model.x[i,j]*margin[i]*seg_prob1[i][segment[j]] for i in range(num_item)) for j in range(num_cust)) 
cost=sum(sum(model.x[i,j]*samplecost[i] for j in range(num_cust)) for i in range(num_item)) 

# Objective: Maximize total expected margin - sample cost
model.Objective = Objective(expr=expected_margin-cost, sense =maximize)


In [8]:
# Leverage GLPK to solve the MIP model
opt=SolverFactory('glpk')
results = opt.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\11416\AppData\Local\Temp\tmpft3mr239.glpk.raw --wglp C:\Users\11416\AppData\Local\Temp\tmp3i449355.glpk.glp
 --cpxlp C:\Users\11416\AppData\Local\Temp\tmpzs4fd975.pyomo.lp
Reading problem data from 'C:\Users\11416\AppData\Local\Temp\tmpzs4fd975.pyomo.lp'...
C:\Users\11416\AppData\Local\Temp\tmpzs4fd975.pyomo.lp:933223: warning: lower bound of variable 'x1' redefined
C:\Users\11416\AppData\Local\Temp\tmpzs4fd975.pyomo.lp:933223: warning: upper bound of variable 'x1' redefined
117376 rows, 116218 columns, 348652 non-zeros
116217 integer variables, all of which are binary
1049440 lines were read
Writing problem data to 'C:\Users\11416\AppData\Local\Temp\tmp3i449355.glpk.glp'...
815843 lines were written
GLPK Integer Optimizer 5.0
117376 rows, 116218 columns, 348652 non-zeros
116217 integer variables, all of which are binary
Preprocessing...
1158 rows, 104370 columns, 208740 non-zeros
104370 intege

In [9]:
#model.pprint()

In [10]:
# Printing solution
print("Obj: Total profit: ", model.Objective())

# Print out the 5 samples for each customer
for j in range(num_cust): #for each customer
    for i in range(num_item): # for each possible sample
        if model.x[i,j]() == 1: #if the sample is assigned to this customer
            print("Customer", cust_id[j], "will receive Item",sku[i])

Obj: Total profit:  15957.657040004979
Customer 0 will receive Item 7
Customer 0 will receive Item 9
Customer 0 will receive Item 14
Customer 0 will receive Item 34
Customer 0 will receive Item 41
Customer 1 will receive Item 24
Customer 1 will receive Item 51
Customer 1 will receive Item 67
Customer 1 will receive Item 94
Customer 1 will receive Item 105
Customer 2 will receive Item 14
Customer 2 will receive Item 30
Customer 2 will receive Item 31
Customer 2 will receive Item 41
Customer 2 will receive Item 58
Customer 3 will receive Item 12
Customer 3 will receive Item 22
Customer 3 will receive Item 28
Customer 3 will receive Item 55
Customer 3 will receive Item 109
Customer 4 will receive Item 6
Customer 4 will receive Item 22
Customer 4 will receive Item 33
Customer 4 will receive Item 60
Customer 4 will receive Item 76
Customer 5 will receive Item 24
Customer 5 will receive Item 67
Customer 5 will receive Item 78
Customer 5 will receive Item 94
Customer 5 will receive Item 105
C

Customer 80 will receive Item 70
Customer 80 will receive Item 71
Customer 80 will receive Item 98
Customer 81 will receive Item 22
Customer 81 will receive Item 33
Customer 81 will receive Item 60
Customer 81 will receive Item 76
Customer 81 will receive Item 103
Customer 82 will receive Item 3
Customer 82 will receive Item 12
Customer 82 will receive Item 25
Customer 82 will receive Item 30
Customer 82 will receive Item 52
Customer 83 will receive Item 7
Customer 83 will receive Item 9
Customer 83 will receive Item 14
Customer 83 will receive Item 34
Customer 83 will receive Item 41
Customer 84 will receive Item 8
Customer 84 will receive Item 35
Customer 84 will receive Item 62
Customer 84 will receive Item 81
Customer 84 will receive Item 89
Customer 85 will receive Item 21
Customer 85 will receive Item 22
Customer 85 will receive Item 30
Customer 85 will receive Item 71
Customer 85 will receive Item 98
Customer 86 will receive Item 17
Customer 86 will receive Item 21
Customer 86 w

Customer 145 will receive Item 61
Customer 145 will receive Item 104
Customer 146 will receive Item 14
Customer 146 will receive Item 31
Customer 146 will receive Item 41
Customer 146 will receive Item 58
Customer 146 will receive Item 98
Customer 147 will receive Item 7
Customer 147 will receive Item 9
Customer 147 will receive Item 14
Customer 147 will receive Item 34
Customer 147 will receive Item 41
Customer 148 will receive Item 7
Customer 148 will receive Item 9
Customer 148 will receive Item 31
Customer 148 will receive Item 34
Customer 148 will receive Item 41
Customer 149 will receive Item 3
Customer 149 will receive Item 12
Customer 149 will receive Item 25
Customer 149 will receive Item 39
Customer 149 will receive Item 52
Customer 150 will receive Item 7
Customer 150 will receive Item 9
Customer 150 will receive Item 14
Customer 150 will receive Item 34
Customer 150 will receive Item 41
Customer 151 will receive Item 12
Customer 151 will receive Item 16
Customer 151 will re

Customer 207 will receive Item 56
Customer 207 will receive Item 83
Customer 208 will receive Item 6
Customer 208 will receive Item 22
Customer 208 will receive Item 76
Customer 208 will receive Item 87
Customer 208 will receive Item 103
Customer 209 will receive Item 12
Customer 209 will receive Item 16
Customer 209 will receive Item 43
Customer 209 will receive Item 70
Customer 209 will receive Item 98
Customer 210 will receive Item 2
Customer 210 will receive Item 11
Customer 210 will receive Item 29
Customer 210 will receive Item 56
Customer 210 will receive Item 83
Customer 211 will receive Item 8
Customer 211 will receive Item 35
Customer 211 will receive Item 62
Customer 211 will receive Item 81
Customer 211 will receive Item 89
Customer 212 will receive Item 19
Customer 212 will receive Item 27
Customer 212 will receive Item 73
Customer 212 will receive Item 81
Customer 212 will receive Item 100
Customer 213 will receive Item 0
Customer 213 will receive Item 27
Customer 213 wil

Customer 270 will receive Item 103
Customer 271 will receive Item 19
Customer 271 will receive Item 46
Customer 271 will receive Item 73
Customer 271 will receive Item 81
Customer 271 will receive Item 100
Customer 272 will receive Item 7
Customer 272 will receive Item 9
Customer 272 will receive Item 14
Customer 272 will receive Item 34
Customer 272 will receive Item 41
Customer 273 will receive Item 8
Customer 273 will receive Item 35
Customer 273 will receive Item 62
Customer 273 will receive Item 81
Customer 273 will receive Item 89
Customer 274 will receive Item 12
Customer 274 will receive Item 16
Customer 274 will receive Item 43
Customer 274 will receive Item 70
Customer 274 will receive Item 98
Customer 275 will receive Item 7
Customer 275 will receive Item 11
Customer 275 will receive Item 12
Customer 275 will receive Item 65
Customer 275 will receive Item 92
Customer 276 will receive Item 0
Customer 276 will receive Item 27
Customer 276 will receive Item 35
Customer 276 will

...

Customer 665 will receive Item 98
Customer 666 will receive Item 2
Customer 666 will receive Item 6
Customer 666 will receive Item 29
Customer 666 will receive Item 56
Customer 666 will receive Item 83
Customer 667 will receive Item 14
Customer 667 will receive Item 31
Customer 667 will receive Item 41
Customer 667 will receive Item 58
Customer 667 will receive Item 98
Customer 668 will receive Item 2
Customer 668 will receive Item 11
Customer 668 will receive Item 29
Customer 668 will receive Item 56
Customer 668 will receive Item 83
Customer 669 will receive Item 16
Customer 669 will receive Item 23
Customer 669 will receive Item 50
Customer 669 will receive Item 61
Customer 669 will receive Item 104
Customer 670 will receive Item 24
Customer 670 will receive Item 51
Customer 670 will receive Item 78
Customer 670 will receive Item 94
Customer 670 will receive Item 105
Customer 671 will receive Item 6
Customer 671 will receive Item 11
Customer 671 will receive Item 31
Customer 671 wil

Customer 725 will receive Item 105
Customer 726 will receive Item 3
Customer 726 will receive Item 12
Customer 726 will receive Item 25
Customer 726 will receive Item 30
Customer 726 will receive Item 39
Customer 727 will receive Item 14
Customer 727 will receive Item 17
Customer 727 will receive Item 41
Customer 727 will receive Item 71
Customer 727 will receive Item 98
Customer 728 will receive Item 12
Customer 728 will receive Item 16
Customer 728 will receive Item 43
Customer 728 will receive Item 70
Customer 728 will receive Item 98
Customer 729 will receive Item 14
Customer 729 will receive Item 30
Customer 729 will receive Item 31
Customer 729 will receive Item 58
Customer 729 will receive Item 98
Customer 730 will receive Item 2
Customer 730 will receive Item 11
Customer 730 will receive Item 29
Customer 730 will receive Item 56
Customer 730 will receive Item 83
Customer 731 will receive Item 3
Customer 731 will receive Item 12
Customer 731 will receive Item 25
Customer 731 wil

Customer 856 will receive Item 58
Customer 857 will receive Item 0
Customer 857 will receive Item 8
Customer 857 will receive Item 35
Customer 857 will receive Item 62
Customer 857 will receive Item 81
Customer 858 will receive Item 8
Customer 858 will receive Item 35
Customer 858 will receive Item 62
Customer 858 will receive Item 81
Customer 858 will receive Item 89
Customer 859 will receive Item 3
Customer 859 will receive Item 12
Customer 859 will receive Item 25
Customer 859 will receive Item 30
Customer 859 will receive Item 52
Customer 860 will receive Item 2
Customer 860 will receive Item 6
Customer 860 will receive Item 11
Customer 860 will receive Item 29
Customer 860 will receive Item 38
Customer 861 will receive Item 12
Customer 861 will receive Item 16
Customer 861 will receive Item 43
Customer 861 will receive Item 70
Customer 861 will receive Item 98
Customer 862 will receive Item 2
Customer 862 will receive Item 11
Customer 862 will receive Item 29
Customer 862 will rec

Customer 920 will receive Item 30
Customer 920 will receive Item 52
Customer 921 will receive Item 7
Customer 921 will receive Item 9
Customer 921 will receive Item 14
Customer 921 will receive Item 34
Customer 921 will receive Item 41
Customer 922 will receive Item 0
Customer 922 will receive Item 27
Customer 922 will receive Item 73
Customer 922 will receive Item 81
Customer 922 will receive Item 100
Customer 923 will receive Item 21
Customer 923 will receive Item 30
Customer 923 will receive Item 71
Customer 923 will receive Item 98
Customer 923 will receive Item 103
Customer 924 will receive Item 17
Customer 924 will receive Item 24
Customer 924 will receive Item 44
Customer 924 will receive Item 71
Customer 924 will receive Item 98
Customer 925 will receive Item 7
Customer 925 will receive Item 9
Customer 925 will receive Item 14
Customer 925 will receive Item 34
Customer 925 will receive Item 41
Customer 926 will receive Item 2
Customer 926 will receive Item 11
Customer 926 will 

Customer 992 will receive Item 11
Customer 992 will receive Item 38
Customer 992 will receive Item 65
Customer 992 will receive Item 92
Customer 993 will receive Item 2
Customer 993 will receive Item 6
Customer 993 will receive Item 11
Customer 993 will receive Item 29
Customer 993 will receive Item 38
Customer 994 will receive Item 6
Customer 994 will receive Item 7
Customer 994 will receive Item 11
Customer 994 will receive Item 38
Customer 994 will receive Item 92
Customer 995 will receive Item 2
Customer 995 will receive Item 11
Customer 995 will receive Item 29
Customer 995 will receive Item 56
Customer 995 will receive Item 83
Customer 996 will receive Item 24
Customer 996 will receive Item 51
Customer 996 will receive Item 78
Customer 996 will receive Item 94
Customer 996 will receive Item 105
Customer 997 will receive Item 12
Customer 997 will receive Item 16
Customer 997 will receive Item 43
Customer 997 will receive Item 70
Customer 997 will receive Item 98
Customer 998 will 

In [11]:
# Save the solution into a xlsx file
solution = [[]]*len(cust_id)
for j in range(num_cust):
    solution[j]=[]
    for i in range(num_item):
        if model.x[i,j]() == 1:
            solution[j].append(sku[i])
# Convert list of lists to dataframe
df_solution = pd.DataFrame(solution, columns = ['Sample 1','Sample 2','Sample 3','Sample 4','Sample 5'])
df_cust = pd.DataFrame(cust_id, columns=['Customer ID'])
df_output = pd.concat([df_cust,df_solution], axis=1)
df_output
# Save to excel file
df_output.to_excel('P5_Ming_Chen_solution.xlsx', index=False)

# Part 2: Next Month’s Revenue with Zero Replenishment
### Update itemreceived variable and resolve the problem


Imagine that Birchbox failed to source any new samples next month – so they must make new assortments but also cannot send customers what they just sent them this month. Solve next month’s problem in this scenario. How much profit should they expect?

In [12]:
# Update the itemreceived variable
itemreceived1 = [[]]*len(range(num_cust))
for j in range(num_cust):
    itemreceived1[j]=[]
    for i in range(num_item):
        itemreceived1[j].append(itemreceived[j][i]+model.x[i,j]())

In [17]:
# Update the inventory
inventory1=[]
for i in range(num_item):
    inventory1.append(inventory[i] - sum(model.x[i,j]() for j in range(num_cust)))

In [18]:
# Resolve the problem by replacing itemreceived with itemreceived1
# Declare a concrete model
model1 = ConcreteModel()
num_item = 111 #possible items, (columns)index them by i
num_cust = 1047 #possible customers, (rows) index them by j

# Declare the decision variables
model1.x = Var(range(num_item),range(num_cust), domain=Binary) #Which items are assigned to each customer?

# Constraint 1: each customer receive exactly 5 samples
model1.sample = ConstraintList()
for j in range(num_cust):
    model1.sample.add(expr = sum(model1.x[i,j] for i in range(num_item)) == 5)
    
# Constraint 2: Cannot run out of inventory
model1.inventory1 = ConstraintList()
for i in range(num_item):
    model1.inventory1.add(expr = sum(model1.x[i,j] for j in range(num_cust)) <= inventory1[i])
    
# Constraint 3: Cannot send a customer a sample they’ve already received.
model1.uniquesample = ConstraintList()
for j in range(num_cust):
    for i in range(num_item):
        model1.uniquesample.add(expr = (model1.x[i,j] + itemreceived1[j][i]) <=1)
        
# Objective calculation
expected_margin=sum(sum(model1.x[i,j]*margin[i]*seg_prob1[i][segment[j]] for i in range(num_item)) for j in range(num_cust)) 
cost=sum(sum(model1.x[i,j]*samplecost[i] for j in range(num_cust)) for i in range(num_item)) 

# Objective: Maximize total expected margin - sample cost
model1.Objective = Objective(expr=expected_margin-cost, sense =maximize)

In [19]:
# Solve again!
opt=SolverFactory('glpk')
results = opt.solve(model1, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\11416\AppData\Local\Temp\tmpheqhaa1o.glpk.raw --wglp C:\Users\11416\AppData\Local\Temp\tmpp3pxkzgq.glpk.glp
 --cpxlp C:\Users\11416\AppData\Local\Temp\tmp9584s1l7.pyomo.lp
Reading problem data from 'C:\Users\11416\AppData\Local\Temp\tmp9584s1l7.pyomo.lp'...
C:\Users\11416\AppData\Local\Temp\tmp9584s1l7.pyomo.lp:933223: warning: lower bound of variable 'x1' redefined
C:\Users\11416\AppData\Local\Temp\tmp9584s1l7.pyomo.lp:933223: warning: upper bound of variable 'x1' redefined
117376 rows, 116218 columns, 348652 non-zeros
116217 integer variables, all of which are binary
1049440 lines were read
Writing problem data to 'C:\Users\11416\AppData\Local\Temp\tmpp3pxkzgq.glpk.glp'...
815843 lines were written
GLPK Integer Optimizer 5.0
117376 rows, 116218 columns, 348652 non-zeros
116217 integer variables, all of which are binary
Preprocessing...
1146 rows, 89069 columns, 178138 non-zeros
89069 integer 

In [20]:
#printing solution
print("Obj: Total profit: ", model1.Objective())

#print out the 5 samples for each customer
for j in range(num_cust): #for each customer
    for i in range(num_item): # for each possible sample
        if model1.x[i,j]() == 1: #if the sample is assigned to this customer
            print("Customer", cust_id[j], "will receive Item",sku[i])

Obj: Total profit:  -898.8606092811297
Customer 0 will receive Item 4
Customer 0 will receive Item 5
Customer 0 will receive Item 8
Customer 0 will receive Item 50
Customer 0 will receive Item 68
Customer 1 will receive Item 13
Customer 1 will receive Item 22
Customer 1 will receive Item 40
Customer 1 will receive Item 58
Customer 1 will receive Item 70
Customer 2 will receive Item 1
Customer 2 will receive Item 3
Customer 2 will receive Item 4
Customer 2 will receive Item 24
Customer 2 will receive Item 88
Customer 3 will receive Item 23
Customer 3 will receive Item 50
Customer 3 will receive Item 53
Customer 3 will receive Item 80
Customer 3 will receive Item 107
Customer 4 will receive Item 37
Customer 4 will receive Item 48
Customer 4 will receive Item 53
Customer 4 will receive Item 64
Customer 4 will receive Item 107
Customer 5 will receive Item 13
Customer 5 will receive Item 22
Customer 5 will receive Item 31
Customer 5 will receive Item 43
Customer 5 will receive Item 70
Custo

Customer 72 will receive Item 96
Customer 72 will receive Item 108
Customer 73 will receive Item 5
Customer 73 will receive Item 23
Customer 73 will receive Item 32
Customer 73 will receive Item 50
Customer 73 will receive Item 104
Customer 74 will receive Item 6
Customer 74 will receive Item 16
Customer 74 will receive Item 43
Customer 74 will receive Item 100
Customer 74 will receive Item 110
Customer 75 will receive Item 5
Customer 75 will receive Item 23
Customer 75 will receive Item 48
Customer 75 will receive Item 50
Customer 75 will receive Item 104
Customer 76 will receive Item 6
Customer 76 will receive Item 16
Customer 76 will receive Item 38
Customer 76 will receive Item 100
Customer 76 will receive Item 110
Customer 77 will receive Item 21
Customer 77 will receive Item 46
Customer 77 will receive Item 69
Customer 77 will receive Item 96
Customer 77 will receive Item 108
Customer 78 will receive Item 10
Customer 78 will receive Item 33
Customer 78 will receive Item 37
Custom

Customer 131 will receive Item 23
Customer 131 will receive Item 48
Customer 131 will receive Item 50
Customer 131 will receive Item 104
Customer 132 will receive Item 46
Customer 132 will receive Item 69
Customer 132 will receive Item 73
Customer 132 will receive Item 96
Customer 132 will receive Item 108
Customer 133 will receive Item 8
Customer 133 will receive Item 50
Customer 133 will receive Item 58
Customer 133 will receive Item 68
Customer 133 will receive Item 88
Customer 134 will receive Item 6
Customer 134 will receive Item 16
Customer 134 will receive Item 38
Customer 134 will receive Item 92
Customer 134 will receive Item 100
Customer 135 will receive Item 4
Customer 135 will receive Item 29
Customer 135 will receive Item 37
Customer 135 will receive Item 48
Customer 135 will receive Item 77
Customer 136 will receive Item 10
Customer 136 will receive Item 29
Customer 136 will receive Item 39
Customer 136 will receive Item 79
Customer 136 will receive Item 84
Customer 137 w

Customer 232 will receive Item 94
Customer 232 will receive Item 108
Customer 233 will receive Item 4
Customer 233 will receive Item 23
Customer 233 will receive Item 50
Customer 233 will receive Item 68
Customer 233 will receive Item 88
Customer 234 will receive Item 4
Customer 234 will receive Item 5
Customer 234 will receive Item 8
Customer 234 will receive Item 50
Customer 234 will receive Item 88
Customer 235 will receive Item 8
Customer 235 will receive Item 23
Customer 235 will receive Item 50
Customer 235 will receive Item 68
Customer 235 will receive Item 88
Customer 236 will receive Item 8
Customer 236 will receive Item 23
Customer 236 will receive Item 50
Customer 236 will receive Item 68
Customer 236 will receive Item 88
Customer 237 will receive Item 4
Customer 237 will receive Item 29
Customer 237 will receive Item 37
Customer 237 will receive Item 48
Customer 237 will receive Item 77
Customer 238 will receive Item 8
Customer 238 will receive Item 16
Customer 238 will rec

Customer 281 will receive Item 107
Customer 282 will receive Item 10
Customer 282 will receive Item 33
Customer 282 will receive Item 37
Customer 282 will receive Item 64
Customer 282 will receive Item 107
Customer 283 will receive Item 29
Customer 283 will receive Item 39
Customer 283 will receive Item 56
Customer 283 will receive Item 79
Customer 283 will receive Item 84
Customer 284 will receive Item 2
Customer 284 will receive Item 29
Customer 284 will receive Item 46
Customer 284 will receive Item 88
Customer 284 will receive Item 100
Customer 285 will receive Item 16
Customer 285 will receive Item 43
Customer 285 will receive Item 92
Customer 285 will receive Item 100
Customer 285 will receive Item 110
Customer 286 will receive Item 16
Customer 286 will receive Item 23
Customer 286 will receive Item 50
Customer 286 will receive Item 58
Customer 286 will receive Item 68
Customer 287 will receive Item 29
Customer 287 will receive Item 39
Customer 287 will receive Item 56
Customer 2

...

Customer 751 will receive Item 92
Customer 751 will receive Item 100
Customer 751 will receive Item 110
Customer 752 will receive Item 7
Customer 752 will receive Item 12
Customer 752 will receive Item 39
Customer 752 will receive Item 87
Customer 752 will receive Item 88
Customer 753 will receive Item 4
Customer 753 will receive Item 29
Customer 753 will receive Item 37
Customer 753 will receive Item 48
Customer 753 will receive Item 77
Customer 754 will receive Item 3
Customer 754 will receive Item 4
Customer 754 will receive Item 24
Customer 754 will receive Item 44
Customer 754 will receive Item 88
Customer 755 will receive Item 11
Customer 755 will receive Item 21
Customer 755 will receive Item 46
Customer 755 will receive Item 69
Customer 755 will receive Item 96
Customer 756 will receive Item 46
Customer 756 will receive Item 69
Customer 756 will receive Item 73
Customer 756 will receive Item 96
Customer 756 will receive Item 108
Customer 757 will receive Item 10
Customer 757 wi

Customer 804 will receive Item 43
Customer 804 will receive Item 67
Customer 804 will receive Item 70
Customer 805 will receive Item 7
Customer 805 will receive Item 12
Customer 805 will receive Item 16
Customer 805 will receive Item 39
Customer 805 will receive Item 87
Customer 806 will receive Item 1
Customer 806 will receive Item 50
Customer 806 will receive Item 67
Customer 806 will receive Item 94
Customer 806 will receive Item 108
Customer 807 will receive Item 2
Customer 807 will receive Item 39
Customer 807 will receive Item 44
Customer 807 will receive Item 88
Customer 807 will receive Item 100
Customer 808 will receive Item 22
Customer 808 will receive Item 40
Customer 808 will receive Item 43
Customer 808 will receive Item 58
Customer 808 will receive Item 70
Customer 809 will receive Item 10
Customer 809 will receive Item 29
Customer 809 will receive Item 56
Customer 809 will receive Item 79
Customer 809 will receive Item 84
Customer 810 will receive Item 5
Customer 810 wil

Customer 895 will receive Item 29
Customer 895 will receive Item 39
Customer 895 will receive Item 56
Customer 895 will receive Item 79
Customer 895 will receive Item 84
Customer 896 will receive Item 10
Customer 896 will receive Item 33
Customer 896 will receive Item 37
Customer 896 will receive Item 64
Customer 896 will receive Item 107
Customer 897 will receive Item 4
Customer 897 will receive Item 8
Customer 897 will receive Item 31
Customer 897 will receive Item 50
Customer 897 will receive Item 88
Customer 898 will receive Item 22
Customer 898 will receive Item 40
Customer 898 will receive Item 43
Customer 898 will receive Item 58
Customer 898 will receive Item 70
Customer 899 will receive Item 29
Customer 899 will receive Item 39
Customer 899 will receive Item 56
Customer 899 will receive Item 79
Customer 899 will receive Item 84
Customer 900 will receive Item 3
Customer 900 will receive Item 4
Customer 900 will receive Item 24
Customer 900 will receive Item 44
Customer 900 will

Customer 945 will receive Item 88
Customer 945 will receive Item 100
Customer 946 will receive Item 4
Customer 946 will receive Item 23
Customer 946 will receive Item 31
Customer 946 will receive Item 68
Customer 946 will receive Item 88
Customer 947 will receive Item 29
Customer 947 will receive Item 44
Customer 947 will receive Item 88
Customer 947 will receive Item 92
Customer 947 will receive Item 100
Customer 948 will receive Item 5
Customer 948 will receive Item 23
Customer 948 will receive Item 32
Customer 948 will receive Item 48
Customer 948 will receive Item 50
Customer 949 will receive Item 13
Customer 949 will receive Item 22
Customer 949 will receive Item 43
Customer 949 will receive Item 67
Customer 949 will receive Item 70
Customer 950 will receive Item 7
Customer 950 will receive Item 34
Customer 950 will receive Item 39
Customer 950 will receive Item 70
Customer 950 will receive Item 87
Customer 951 will receive Item 2
Customer 951 will receive Item 29
Customer 951 wil

Customer 1019 will receive Item 96
Customer 1019 will receive Item 108
Customer 1020 will receive Item 10
Customer 1020 will receive Item 39
Customer 1020 will receive Item 56
Customer 1020 will receive Item 79
Customer 1020 will receive Item 84
Customer 1021 will receive Item 29
Customer 1021 will receive Item 44
Customer 1021 will receive Item 67
Customer 1021 will receive Item 88
Customer 1021 will receive Item 100
Customer 1022 will receive Item 1
Customer 1022 will receive Item 13
Customer 1022 will receive Item 50
Customer 1022 will receive Item 67
Customer 1022 will receive Item 94
Customer 1023 will receive Item 3
Customer 1023 will receive Item 4
Customer 1023 will receive Item 24
Customer 1023 will receive Item 44
Customer 1023 will receive Item 88
Customer 1024 will receive Item 2
Customer 1024 will receive Item 29
Customer 1024 will receive Item 44
Customer 1024 will receive Item 67
Customer 1024 will receive Item 88
Customer 1025 will receive Item 3
Customer 1025 will rece